# Kinetics 동영상 데이터셋을 데이터로더로 구현

In [1]:
import os
from PIL import Image
import csv
import numpy as np

import torch
import torch.utils.data
from torch import nn

import torchvision

## Kinetics 동영상 데이터셋으로 ECO 데이터로더작성


In [3]:
def make_datapath_list(root_path):
    video_list = list()

    class_list = os.listdir(root_path)

    for class_list_i in (class_list):
        class_path = os.path.join(root_path, class_list_i)

        for file_name in os.listdir(class_path):
            name, ext = os.path.splitext(file_name)

            if ext == ".mp4":
                continue
            
            video_img_directory_path = os.path.join(class_path, name)
            video_list.append(video_img_directory_path)

    return video_list

root_path = "./data/kinetics_videos/"
video_list = make_datapath_list(root_path)
print(video_list[0])
print(video_list[1])

./data/kinetics_videos/bungee jumping/40c7413c-cda1-4e7c-bc53-3b5dc44db082
./data/kinetics_videos/bungee jumping/a9bdfaa1-81de-4bbb-8aac-473c79c1096f


In [5]:
class VideoTransform():
    def __init__(self, resize, crop_size, mean, std):
        self.data_transform = {
            "train":torchvision.transforms.Compose([
                GroupResize(int(resize)),
                GroupCenterCrop(crop_size),
                GroupToTensor(),
                GroupImgNormalize(mean, std),
                Stack()
            ]),
            "val":torchvision.transforms.Compose([
                GroupResize(int(resize)),
                GroupCenterCrop(crop_size),
                GroupToTensor(),
                GroupImgNormalize(mean, std),
                Stack()
            ])
        }
    
    def __call_(self, img_group, phase):
        return self.data_transform[phase](img_group)

In [ ]:
class GroupResize():
    def __init__(self, resize, interpolation=Image.BILINEAR):
        self.rescaler = torchvision.transforms.Resize(resize, interpolation)

    def __call__(self, img_group):
        return [self.rescaler(img) for img in img_group]

class GroupCenterCrop():
    def __init__(self, crop_size):
        self.ccrop = torchvision.transforms.CenterCrop(crop_size)

    def __call__(self, img_group):
        return [self.ccrop(img) for img in img_group]

class GroupToTensor():
    def __init__(self):
        self.to_tensor = torchvision.transforms.ToTensor()

    def __call__(self, img_group):
        return [self.to_tensor(img)*255 for img in img_group]

class GroupImgNormalize():
    def __init__(self, mean, std):
        self.normalize = torchvision.transforms.Normalize(mean, std)

    def __call__(self, img_group):
        return [self.normalize(img) for img in img_group]

class Stack():
    def __call__(self, img_group):
        ret = torch.cat([(x.filp(dims=[0])).unsqueeze(dim=0) for x in img_group], dim=0)

        return ret